In [1]:
import os
import pandas as pd
import numpy as np
import collections
from sklearn.preprocessing import StandardScaler


수비점수 = 수비율 * 수비이닝 * 포지션 가중치
March
포지션 가중치
- 포수 = 8
- 내야수
  = 1루수 2
  = 3루수 4
  = 2루수 6
  = 유격수 7
- 외야수
  = 좌익수 2
  = 우익수 2.5
  = 중견수 3.5
  
kWAR 포지션보정점수 
MinMaxScaler
포수 = 1.0     ## 1.0
1루수 = 0.0    ## -1.0
2루수 = 0.6    ## 0.2
3루수 = 0.6    ## 0.2
유격수 = 0.8   ## 0.6
좌익수 = 0.2   ## -0.6
중견수 = 0.4   ## -0.2
우익수 = 0.2   ## -0.6
  
FPCT (수비율or 수비성공률) - (자살+보살) / (자살+보살+실책) 

In [68]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율')
dfn_1 = pd.read_csv('2019_전체.csv')
dfn_1.index = dfn_1['선수명'] + dfn_1['팀명']
del dfn_1['선수명']
dfn_1 = dfn_1.drop(columns = ['G', 'GS', 'PKO', 'PO', 'A', 'DP', 'PB', 'SB', 'CS', 'CS%'])
dfn_1 = dfn_1.replace('-', '0')
dfn_1

,팀명,POS,IP,E,FPCT
강경학한화,한화,2루수,51.00,1,0.971
강경학한화,한화,1루수,69.00,0,1.000
강경학한화,한화,유격수,153.00,3,0.965
강로한롯데,롯데,3루수,183.33,3,0.950
강로한롯데,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균KT,KT,1루수,8.67,0,1.000
황재균KT,KT,유격수,54.00,1,0.968
황재균KT,KT,3루수,964.00,10,0.970
황진수롯데,롯데,2루수,1.00,0,0


In [70]:
## integer로 타입변환
FPCT_list = []
for a in dfn_1['FPCT'] :
    FPCT_list.append(int(a.replace('.', '').strip()) / 1000)
dfn_1['FPCT'] = FPCT_list
dfn_1

,팀명,POS,IP,E,FPCT
강경학한화,한화,2루수,51.00,1,0.971
강경학한화,한화,1루수,69.00,0,1.000
강경학한화,한화,유격수,153.00,3,0.965
강로한롯데,롯데,3루수,183.33,3,0.950
강로한롯데,롯데,유격수,261.67,12,0.911
...,...,...,...,...,...
황재균KT,KT,1루수,8.67,0,1.000
황재균KT,KT,유격수,54.00,1,0.968
황재균KT,KT,3루수,964.00,10,0.970
황진수롯데,롯데,2루수,1.00,0,0.000


#### kWAR 포지션 보정점수 기준

In [71]:
## 수비율 * 수비이닝 * 포지션별 가중치
D_score_list = []
for a in range(0, len(dfn_1)) :
    if dfn_1.iloc[a, 1] == '1루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '3루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '2루수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.6, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '유격수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.8, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '좌익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '중견수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.4, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '우익수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 0.2, 3)
        D_score_list.append(D_score)
    elif dfn_1.iloc[a, 1] == '포수' :
        D_score = round(dfn_1.iloc[a, 4] * dfn_1.iloc[a, 2] * 1.0, 3)
        D_score_list.append(D_score)
dfn_1['kWAR_score'] = D_score_list
dfn_1

,팀명,POS,IP,E,FPCT,kWAR_score
강경학한화,한화,2루수,51.00,1,0.971,29.713
강경학한화,한화,1루수,69.00,0,1.000,0.000
강경학한화,한화,유격수,153.00,3,0.965,118.116
강로한롯데,롯데,3루수,183.33,3,0.950,104.498
강로한롯데,롯데,유격수,261.67,12,0.911,190.705
...,...,...,...,...,...,...
황재균KT,KT,1루수,8.67,0,1.000,0.000
황재균KT,KT,유격수,54.00,1,0.968,41.818
황재균KT,KT,3루수,964.00,10,0.970,561.048
황진수롯데,롯데,2루수,1.00,0,0.000,0.000


In [72]:
dfn_11 = dfn_1.reset_index()
dfn_11

,index,팀명,POS,IP,E,FPCT,kWAR_score
0,강경학한화,한화,2루수,51.00,1,0.971,29.713
1,강경학한화,한화,1루수,69.00,0,1.000,0.000
2,강경학한화,한화,유격수,153.00,3,0.965,118.116
3,강로한롯데,롯데,3루수,183.33,3,0.950,104.498
4,강로한롯데,롯데,유격수,261.67,12,0.911,190.705
...,...,...,...,...,...,...,...
602,황재균KT,KT,1루수,8.67,0,1.000,0.000
603,황재균KT,KT,유격수,54.00,1,0.968,41.818
604,황재균KT,KT,3루수,964.00,10,0.970,561.048
605,황진수롯데,롯데,2루수,1.00,0,0.000,0.000


In [73]:
dfn_12 = dfn_11.append({'index' : 0}, ignore_index = True)
dfn_12

,index,팀명,POS,IP,E,FPCT,kWAR_score
0,강경학한화,한화,2루수,51.00,1.0,0.971,29.713
1,강경학한화,한화,1루수,69.00,0.0,1.000,0.000
2,강경학한화,한화,유격수,153.00,3.0,0.965,118.116
3,강로한롯데,롯데,3루수,183.33,3.0,0.950,104.498
4,강로한롯데,롯데,유격수,261.67,12.0,0.911,190.705
...,...,...,...,...,...,...,...
603,황재균KT,KT,유격수,54.00,1.0,0.968,41.818
604,황재균KT,KT,3루수,964.00,10.0,0.970,561.048
605,황진수롯데,롯데,2루수,1.00,0.0,0.000,0.000
606,황진수롯데,롯데,3루수,9.33,0.0,1.000,5.598


In [74]:
dfn_12.head(10)

,index,팀명,POS,IP,E,FPCT,kWAR_score
0,강경학한화,한화,2루수,51.00,1.0,0.971,29.713
1,강경학한화,한화,1루수,69.00,0.0,1.000,0.000
2,강경학한화,한화,유격수,153.00,3.0,0.965,118.116
3,강로한롯데,롯데,3루수,183.33,3.0,0.950,104.498
4,강로한롯데,롯데,유격수,261.67,12.0,0.911,190.705
5,강로한롯데,롯데,2루수,264.67,5.0,0.973,154.514
6,강민국KT,KT,3루수,31.67,1.0,0.917,17.425
7,강민국KT,KT,2루수,38.00,3.0,0.864,19.699
8,강민국KT,KT,유격수,176.00,5.0,0.953,134.182
9,강민호삼성,삼성,포수,842.00,8.0,0.989,832.738


In [75]:
ID = '강경학한화'
DC = 0
DC_list = []
ID_list = []
for a in range(0, len(dfn_12)) :
    ID_1 = dfn_12.iloc[a, 0]
    if ID_1 == ID :
        DC += dfn_12.iloc[a, 6]
    else :
        ID_list.append(ID)
        DC_list.append(DC)
        DC = 0
        ID = dfn_12.iloc[a, 0]
        DC += dfn_12.iloc[a, 6]
    

In [76]:
dfn_2 = pd.DataFrame()
dfn_2['index'] = dfn_11['index'].unique()
dfn_2['DC'] = DC_list
dfn_2

,index,DC
0,강경학한화,147.829
1,강로한롯데,449.717
2,강민국KT,171.306
3,강민호삼성,832.738
4,강백호KT,127.421
...,...,...
279,홍창기LG,13.400
280,황대인KIA,1.200
281,황윤호KIA,177.799
282,황재균KT,602.866


In [77]:
## 타격기록 없는거
dfn_3 = dfn_2[dfn_2['index'] != '김태근두산']
dfn_3 = dfn_3[dfn_3['index'] != '전민재두산']
dfn_3

,index,DC
0,강경학한화,147.829
1,강로한롯데,449.717
2,강민국KT,171.306
3,강민호삼성,832.738
4,강백호KT,127.421
...,...,...
279,홍창기LG,13.400
280,황대인KIA,1.200
281,황윤호KIA,177.799
282,황재균KT,602.866


In [79]:
os.chdir(r'C:\Users\jaij2\OneDrive\바탕 화면\논문\야구데이터\수비율\전처리\220221')
dfn_3.to_csv('2019_전선수_수비기여도.csv', encoding = 'cp949', index = False)